In [1]:
%load_ext autoreload

In [ ]:
""" 
Training set for NATO phonetic alphabets.

Each alphabet is repeated 20 times. 16 instances of each alphabet are used for training and 4 instances for validation. 
Test sets are the entire RAINBOW and GRANDFATHER passages.

0: Alfa
1: Bravo
2: Charlie
3: Delta
4: Echo
5: Foxtrot
6: Golf
7: Hotel
8: India
9: Juliette
10: Kilo
11: Lima
12: Mike
13: November
14: Oscar
15: Papa
16: Quebec
17: Romeo
18: Sierra
19: Tango
20: Uniform
21: Victor
22: Whiskey
23: X-ray
24: Yankee
25: Zulu

DATA is given in a numpy array of dimensions (520, 22, 7500) - (26 alphabets each repeated 20 times, 22 channels, 7500 time samples).
Raw data was filtered using 3rd order Butterworth bandpass filter between 80 and 1000 Hertz.

RUN this file before running Part2_NATOAlphabets_GrandfatherRnn.ipynb and Part2_NATOAlphabets_RainbowRnn.ipynb as this files saves the model weights for testing.
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from manifoldRnn import spdNN
from manifoldRnn import optimizers 
from manifoldRnn import trainTest
from manifoldRnn import spdRnn

In [3]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [4]:
dev = "cuda:0" 
device = torch.device(dev)

In [5]:
numberAlphabets = 26
trialsPerAlphabet = 20
numberTrials = numberAlphabets * trialsPerAlphabet
numberChannels = 22
windowLength = 7500

In [6]:
subjectNumber = 4
subject = "Subject" + str(subjectNumber)

In [7]:
DATA = np.load("Experiment2/" + subject + "/trainSet.npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

In [8]:
slicedMatrices = np.zeros((numberAlphabets * trialsPerAlphabet, 46, numberChannels, numberChannels))
for j in range(numberAlphabets * trialsPerAlphabet):
    for i in range(46):
        where = i * 150 + 300
        start = where - 300
        End = where + 450
        slicedMatrices[j, i] = 1/750 * DATA[j, :, start:End] @ DATA[j, :, start:End].T

In [9]:
labelsByAlphabet = np.array([[i] * trialsPerAlphabet for i in range(numberAlphabets)]).reshape(numberTrials)

Indices =  {}
for i in range(numberAlphabets):
    Indices[i] = []
for i in range(len(labelsByAlphabet)):
    Indices[labelsByAlphabet[i]].append(i)

covariancesLabels = np.zeros((numberAlphabets, trialsPerAlphabet, 46, numberChannels, numberChannels))
for i in range(numberAlphabets):
    for j in range(trialsPerAlphabet):
        covariancesLabels[i, j] = slicedMatrices[Indices[i][j]] 

In [10]:
trainFeatures = np.zeros((numberAlphabets * 16, 46, numberChannels, numberChannels))
trainLabels = np.zeros((numberAlphabets * 16))
count = 0
for i in range(numberAlphabets):
    trainFeatures[count:count + 4] = covariancesLabels[i, :4]
    trainFeatures[count + 4:count + 8] = covariancesLabels[i, 5:9]
    trainFeatures[count + 8:count + 12] = covariancesLabels[i, 10:14]
    trainFeatures[count + 12:count + 16] = covariancesLabels[i, 15:19]
    trainLabels[count:count + 16] = [i] * 16
    count += 16

valFeatures = np.zeros((numberAlphabets * 4, 46, numberChannels, numberChannels))
valLabels = np.zeros((numberAlphabets * 4))
count = 0
for i in range(numberAlphabets):
    valFeatures[count] = covariancesLabels[i,4]
    valFeatures[count + 1] = covariancesLabels[i, 9]
    valFeatures[count + 2] = covariancesLabels[i, 14]
    valFeatures[count + 3] = covariancesLabels[i, 19]
    valLabels[count:count + 4] = [i] * 4
    count += 4

In [11]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
valDataset = BaseDataset(valFeatures, valLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
valDataloader = DataLoader(valDataset, batch_size = 32, shuffle = False)

In [12]:
numberEpochs = 100

model = spdRnn.spdRnnNet(numberAlphabets).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)
lossFunction = nn.CrossEntropyLoss()
cnnOptimizer = optimizers.StiefelOptim(model.CNN.parameters(), lr = 0.05)
rnnOptimizer = optim.Adam(model.RNN.parameters(), lr = 0.001, weight_decay = 1e-3)

145788


In [13]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, cnnOptimizer, rnnOptimizer, lossFunction)
    valLoss, valAccuracy = trainTest.testOperation(model, device, valDataloader, lossFunction)
    if maxValue < valAccuracy:
        maxValue = valAccuracy
        torch.save(model.state_dict(), 'Experiment2/' + subject + '/rnn.pt')
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Val loss: {valLoss:.4f}, Val accuracy: {valAccuracy:.2f}%')
print(maxValue)

Epoch: 1/100, Training loss: 0.1028, Training accuracy: 3.85%, Val loss: 0.1240, Val accuracy: 6.73%
Epoch: 2/100, Training loss: 0.1008, Training accuracy: 7.21%, Val loss: 0.1225, Val accuracy: 13.46%
Epoch: 3/100, Training loss: 0.0975, Training accuracy: 11.54%, Val loss: 0.1151, Val accuracy: 9.62%
Epoch: 4/100, Training loss: 0.0917, Training accuracy: 14.42%, Val loss: 0.1095, Val accuracy: 24.04%
Epoch: 5/100, Training loss: 0.0851, Training accuracy: 24.76%, Val loss: 0.0966, Val accuracy: 27.88%
Epoch: 6/100, Training loss: 0.0776, Training accuracy: 23.32%, Val loss: 0.0858, Val accuracy: 26.92%
Epoch: 7/100, Training loss: 0.0718, Training accuracy: 29.57%, Val loss: 0.0885, Val accuracy: 23.08%
Epoch: 8/100, Training loss: 0.0646, Training accuracy: 32.45%, Val loss: 0.0863, Val accuracy: 25.96%
Epoch: 9/100, Training loss: 0.0656, Training accuracy: 33.17%, Val loss: 0.0748, Val accuracy: 36.54%
Epoch: 10/100, Training loss: 0.0552, Training accuracy: 38.70%, Val loss: 0.

In [14]:
%autoreload